# PN Junction Diode

## Understanding the Fundamental Semiconductor Device

The PN junction diode is the most fundamental semiconductor device and forms the basis for understanding more complex devices like transistors and solar cells.

**Learning Objectives:**
- Understand PN junction physics
- Simulate equilibrium band diagrams with PADRE
- Analyze forward and reverse bias characteristics
- Extract diode parameters from simulated I-V curves

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## 1. PN Junction Physics

### 1.1 Structure

A PN junction consists of:
- **P-region**: Semiconductor doped with acceptors (holes are majority carriers)
- **N-region**: Semiconductor doped with donors (electrons are majority carriers)
- **Depletion region**: The space charge region at the junction

```
    P-region          |    N-region
  (Acceptors Na)      |  (Donors Nd)
                      |
  Holes (majority)    |  Electrons (majority)
  Electrons (minority)|  Holes (minority)
                      |
         <-- Depletion Region -->
```

### 1.2 Key Parameters

- **Built-in potential** ($V_{bi}$): The potential barrier at equilibrium
  $$V_{bi} = \frac{kT}{q} \ln\left(\frac{N_a N_d}{n_i^2}\right)$$

- **Depletion width** ($W$): Width of the space charge region
  $$W = \sqrt{\frac{2\epsilon_s}{q}\left(\frac{1}{N_a} + \frac{1}{N_d}\right)(V_{bi} - V)}$$

- **Diode current** (ideal):
  $$I = I_s\left(e^{qV/kT} - 1\right)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_pn_diode

# Physical constants
q = 1.6e-19      # Elementary charge (C)
kT = 0.0259      # Thermal voltage at 300K (eV)
ni = 1.5e10      # Intrinsic carrier concentration for Si (cm^-3)
eps_si = 11.7 * 8.85e-14  # Silicon permittivity (F/cm)

# Calculate built-in potential
Na = 1e17  # Acceptor concentration (cm^-3)
Nd = 1e17  # Donor concentration (cm^-3)

Vbi = kT * np.log(Na * Nd / ni**2)
print(f"Built-in potential Vbi = {Vbi:.3f} V")

# Calculate depletion width at equilibrium
W = np.sqrt(2 * eps_si / q * (1/Na + 1/Nd) * Vbi) * 1e4  # Convert to microns
print(f"Depletion width W = {W:.4f} um")

---

## 2. Creating and Running a PN Diode Simulation

Let's create a basic PN diode and run PADRE to examine its equilibrium properties.

In [ ]:
# Create a PN diode with symmetric doping
sim_eq = create_pn_diode(
    # Geometry
    length=2.0,              # 2 um total length
    width=1.0,               # 1 um width
    junction_position=0.5,   # Junction at center
    
    # Mesh
    nx=200,                  # Fine mesh for accurate results
    ny=3,                    # Minimal y-mesh (quasi-1D)
    
    # Doping
    p_doping=1e17,           # P-region: 1e17 cm^-3
    n_doping=1e17,           # N-region: 1e17 cm^-3
    
    # Models
    temperature=300,         # Room temperature
    srh=True,               # SRH recombination
    conmob=True,            # Concentration-dependent mobility
    
    # Output - log band diagram at equilibrium
    log_bands_eq=True
)

print("PN Diode Simulation Created")
print("="*40)
print(f"Device length: 2.0 um")
print(f"P-doping: 1e17 cm^-3")
print(f"N-doping: 1e17 cm^-3")
print(f"Junction position: 1.0 um (center)")

In [ ]:
# View the generated PADRE input deck
print("Generated PADRE Input Deck:")
print("="*60)
print(sim_eq.generate_deck())

In [ ]:
# Run the PADRE simulation
print("Running PADRE simulation...")
result = sim_eq.run()

if result.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_eq.working_dir}")
else:
    print("Simulation failed!")
    print(result.stderr)

---

## 3. Equilibrium Band Diagram

At equilibrium (zero bias), the Fermi level is constant throughout the device. The band bending creates the built-in potential barrier.

Let's plot the simulated band diagram from PADRE.

In [ ]:
# Plot the equilibrium band diagram from PADRE simulation
print("\nAvailable outputs:")
print(sim_eq.outputs.summary())

In [ ]:
# Plot band diagram using the built-in plotting method
sim_eq.plot_band_diagram(title="PN Junction Equilibrium Band Diagram (PADRE Simulation)")

In [ ]:
# Get the raw band data for custom plotting
ec_data = sim_eq.outputs.get("cbeq")  # Conduction band
ev_data = sim_eq.outputs.get("vbeq")  # Valence band

if ec_data is not None and ev_data is not None:
    plt.figure(figsize=(10, 6))
    plt.plot(ec_data.x, ec_data.y, 'b-', linewidth=2, label='Ec (Conduction Band)')
    plt.plot(ev_data.x, ev_data.y, 'r-', linewidth=2, label='Ev (Valence Band)')
    
    # Add Fermi level (at equilibrium, it's constant - use midgap as reference)
    Ef = (ec_data.y[0] + ev_data.y[0]) / 2  # Approximate Ef
    plt.axhline(y=0, color='g', linestyle='--', linewidth=1.5, label='Ef (Fermi Level)')
    
    # Mark the junction
    junction = 1.0  # um
    plt.axvline(x=junction, color='gray', linestyle=':', alpha=0.5, label='Junction')
    
    plt.xlabel('Position (μm)', fontsize=12)
    plt.ylabel('Energy (eV)', fontsize=12)
    plt.title('PN Junction Equilibrium Band Diagram (PADRE Simulation)', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)
    
    # Add region labels
    plt.annotate('P-region', xy=(0.3, plt.ylim()[1]*0.8), fontsize=12)
    plt.annotate('N-region', xy=(1.5, plt.ylim()[1]*0.8), fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate built-in potential from simulation
    Vbi_sim = ec_data.y[0] - ec_data.y[-1]
    print(f"\nSimulated built-in potential: Vbi = {abs(Vbi_sim):.3f} V")
    print(f"Theoretical built-in potential: Vbi = {Vbi:.3f} V")
else:
    print("Band data not available. Check simulation output.")

### Understanding the Band Diagram

The equilibrium band diagram shows:

1. **P-region (left)**: 
   - Fermi level near valence band
   - Bands curved upward toward junction

2. **N-region (right)**:
   - Fermi level near conduction band
   - Bands curved downward toward junction

3. **Junction**:
   - Band bending = built-in potential
   - Depletion region where bands curve

---

## 4. Forward Bias I-V Characteristics

Under forward bias (positive voltage on P-side):
- Barrier height decreases
- Current increases exponentially
- Minority carrier injection increases

Let's simulate the forward bias I-V curve with PADRE.

In [ ]:
# Create diode simulation with forward bias sweep
sim_forward = create_pn_diode(
    length=2.0,
    junction_position=0.5,
    p_doping=1e17,
    n_doping=1e17,
    nx=200,
    ny=3,
    
    # Models
    temperature=300,
    srh=True,
    conmob=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="forward_iv",
    
    # Forward bias sweep: 0 to 0.8V in 0.02V steps
    forward_sweep=(0.0, 0.8, 0.02),
    
    # Also log band diagrams at final bias
    log_bands_eq=True
)

print("Forward Bias Simulation Configured")
print("="*40)
print("Voltage sweep: 0V to 0.8V")
print("Step size: 0.02V")
print("Number of bias points:", int(0.8/0.02) + 1)

In [ ]:
# Run the forward bias simulation
print("Running forward bias simulation...")
result_fwd = sim_forward.run()

if result_fwd.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_fwd.stderr)

In [ ]:
# Plot the simulated I-V characteristic
import os

# Get I-V data from simulation
iv_data = sim_forward.get_iv_data()

# Debug: Show IV data info
print(f"IV Data loaded:")
print(f"  Number of electrodes: {iv_data.num_electrodes}")
print(f"  Number of bias points: {len(iv_data.bias_points)}")

if len(iv_data.bias_points) > 0:
    # Get data from electrode 2 (anode)
    V_sim, I_sim = iv_data.get_iv_data(electrode=2)
    V_sim = np.array(V_sim)
    I_sim = np.abs(np.array(I_sim))
    
    if len(V_sim) == 0:
        # Try electrode 1 instead
        print("  Electrode 2 has no data, trying electrode 1...")
        V_sim, I_sim = iv_data.get_iv_data(electrode=1)
        V_sim = np.array(V_sim)
        I_sim = np.abs(np.array(I_sim))
    
    print(f"  Extracted {len(V_sim)} data points")
    
    if len(V_sim) > 0:
        print(f"  Voltage range: {V_sim.min():.3f} to {V_sim.max():.3f} V")
        print(f"  Current range: {I_sim.min():.3e} to {I_sim.max():.3e} A")
        
        # Create figure with two subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Linear scale plot
        ax1.plot(V_sim, I_sim * 1e6, 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
        ax1.set_xlabel('Voltage (V)', fontsize=12)
        ax1.set_ylabel('Current (μA)', fontsize=12)
        ax1.set_title('Forward I-V (Linear Scale)', fontsize=14)
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Semi-log scale plot
        mask = I_sim > 0
        if np.any(mask):
            ax2.semilogy(V_sim[mask], I_sim[mask], 'b-o', linewidth=2, markersize=4, label='PADRE Simulation')
        ax2.set_xlabel('Voltage (V)', fontsize=12)
        ax2.set_ylabel('Current (A)', fontsize=12)
        ax2.set_title('Forward I-V (Semi-log Scale)', fontsize=14)
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Extract diode parameters
        print("\nSimulated Diode Parameters:")
        print("="*40)
        
        # Find current at specific voltages
        for v_target in [0.5, 0.6, 0.7]:
            idx = np.argmin(np.abs(V_sim - v_target))
            if idx < len(I_sim):
                print(f"I({V_sim[idx]:.2f}V) = {I_sim[idx]:.3e} A")
    else:
        print("  No valid data extracted from electrode 2")
else:
    # Show contents of working directory for debugging
    print("\nNo bias points found. Checking output files...")
    print(f"Working directory: {sim_forward.working_dir}")
    print("Files in working directory:")
    for f in os.listdir(sim_forward.working_dir):
        fpath = os.path.join(sim_forward.working_dir, f)
        fsize = os.path.getsize(fpath)
        print(f"  {f} ({fsize} bytes)")
    
    # Try to show IV file content
    iv_file_path = os.path.join(sim_forward.working_dir, "forward_iv")
    if os.path.exists(iv_file_path):
        print(f"\nIV file content (first 1000 chars):")
        with open(iv_file_path, 'r') as f:
            print(f.read()[:1000])

In [ ]:
# Extract ideality factor from the simulation data
try:
    # Use the linear region (0.3V to 0.6V) for ideality factor extraction
    mask = (V_sim > 0.3) & (V_sim < 0.6) & (I_sim > 0)
    V_fit = V_sim[mask]
    I_fit = I_sim[mask]
    
    if len(V_fit) > 2:
        # Fit ln(I) vs V
        coeffs = np.polyfit(V_fit, np.log(I_fit), 1)
        slope = coeffs[0]
        
        # Ideality factor: n = q/(kT * slope)
        n = 1 / (kT * slope)
        
        # Saturation current from intercept
        Is = np.exp(coeffs[1])
        
        print(f"Extracted ideality factor: n = {n:.2f}")
        print(f"Saturation current: Is = {Is:.3e} A")
        print(f"\nNote: Ideal diode has n=1, real diodes typically have n=1-2")
except Exception as e:
    print(f"Could not extract ideality factor: {e}")

---

## 5. Reverse Bias Characteristics

Under reverse bias (negative voltage on P-side):
- Barrier height increases
- Depletion region widens
- Only small reverse saturation current flows

In [ ]:
# Create diode with reverse bias sweep
sim_reverse = create_pn_diode(
    length=2.0,
    junction_position=0.5,
    p_doping=1e17,
    n_doping=1e17,
    nx=200,
    ny=3,
    
    temperature=300,
    srh=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="reverse_iv",
    
    # Reverse bias sweep: 0 to -5V
    reverse_sweep=(0.0, -5.0, -0.25)
)

print("Reverse Bias Simulation Configured")
print("="*40)
print("Voltage sweep: 0V to -5V")
print("Step size: -0.25V")

In [ ]:
# Run the reverse bias simulation
print("Running reverse bias simulation...")
result_rev = sim_reverse.run()

if result_rev.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")

In [ ]:
# Plot reverse bias characteristics
try:
    iv_rev = sim_reverse.get_iv_data()
    V_rev, I_rev = iv_rev.get_iv_data(electrode=2)
    V_rev = np.array(V_rev)
    I_rev = np.abs(np.array(I_rev))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Current vs reverse bias
    ax1.plot(np.abs(V_rev), I_rev, 'r-o', linewidth=2, markersize=4)
    ax1.set_xlabel('Reverse Bias (V)', fontsize=12)
    ax1.set_ylabel('Reverse Current (A)', fontsize=12)
    ax1.set_title('Reverse I-V Characteristic', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Depletion width vs reverse bias (theoretical)
    V_theoretical = np.linspace(0, 5, 50)
    W_vs_V = np.sqrt(2 * eps_si / q * (1/Na + 1/Nd) * (Vbi + V_theoretical)) * 1e4
    
    ax2.plot(V_theoretical, W_vs_V, 'b-', linewidth=2)
    ax2.set_xlabel('Reverse Bias (V)', fontsize=12)
    ax2.set_ylabel('Depletion Width (μm)', fontsize=12)
    ax2.set_title('Depletion Width vs Reverse Bias (Theoretical)', fontsize=14)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nReverse saturation current: Is ≈ {np.mean(I_rev):.3e} A")
    
except Exception as e:
    print(f"Could not plot reverse I-V: {e}")

---

## 6. Effect of Doping Concentration

Let's explore how doping affects the diode characteristics using PADRE simulations.

In [ ]:
# Simulate diodes with different doping levels
doping_levels = [1e16, 1e17, 1e18]
results = {}

print("Simulating diodes with different doping levels:")
print("="*50)

for doping in doping_levels:
    print(f"\nDoping: {doping:.0e} cm^-3")
    
    sim = create_pn_diode(
        length=2.0,
        junction_position=0.5,
        p_doping=doping,
        n_doping=doping,
        nx=150,
        ny=3,
        temperature=300,
        srh=True,
        log_iv=True,
        iv_file=f"iv_{doping:.0e}",
        forward_sweep=(0.0, 0.8, 0.02)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        # Calculate theoretical Vbi
        Vbi_calc = kT * np.log(doping * doping / ni**2)
        print(f"  Vbi (theoretical) = {Vbi_calc:.3f} V")
        
        try:
            iv_data = sim.get_iv_data()
            V, I = iv_data.get_iv_data(electrode=2)
            results[doping] = (np.array(V), np.abs(np.array(I)))
            print(f"  Simulation successful")
        except:
            print(f"  Could not parse I-V data")
    else:
        print(f"  Simulation failed")

In [ ]:
# Plot comparison of different doping levels
if results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    colors = ['blue', 'green', 'red']
    
    for (doping, (V, I)), color in zip(results.items(), colors):
        label = f'Na=Nd={doping:.0e} cm⁻³'
        ax1.plot(V, I * 1e6, color=color, linewidth=2, label=label)
        ax2.semilogy(V, I, color=color, linewidth=2, label=label)
    
    ax1.set_xlabel('Voltage (V)', fontsize=12)
    ax1.set_ylabel('Current (μA)', fontsize=12)
    ax1.set_title('I-V Characteristics (Linear)', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.set_xlabel('Voltage (V)', fontsize=12)
    ax2.set_ylabel('Current (A)', fontsize=12)
    ax2.set_title('I-V Characteristics (Log Scale)', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Higher doping → Lower saturation current (smaller Is)")
    print("- Higher doping → Higher built-in potential")
    print("- All curves show exponential behavior in forward bias")

---

## 7. Complete Simulation with Band Diagrams Under Bias

Let's create a complete simulation that shows band diagrams at different bias conditions.

In [ ]:
# Complete PN diode characterization with band diagrams
sim_complete = create_pn_diode(
    # Device geometry
    length=2.0,
    width=1.0,
    junction_position=0.5,
    
    # High-resolution mesh
    nx=200,
    ny=3,
    
    # Doping
    p_doping=1e17,
    n_doping=1e17,
    
    # Physical models
    temperature=300,
    srh=True,
    conmob=True,
    fldmob=True,
    
    # Material parameters
    taun0=1e-6,  # Electron lifetime
    taup0=1e-6,  # Hole lifetime
    
    # Output logging
    log_iv=True,
    iv_file="complete_iv",
    log_bands_eq=True,
    log_bands_bias=True,
    
    # Forward bias sweep
    forward_sweep=(0.0, 0.7, 0.1)
)

print("Complete PN Diode Simulation")
print("="*50)
print("Running simulation...")

result_complete = sim_complete.run()

if result_complete.returncode == 0:
    print("Simulation completed successfully!")
    print(f"\nOutput directory: {sim_complete.working_dir}")
    print("\nOutputs generated:")
    print(sim_complete.outputs.summary())
else:
    print("Simulation failed!")

In [ ]:
# Plot all band diagram sets (equilibrium and biased)
try:
    sim_complete.plot_band_diagram(title="Band Diagrams at Different Bias Conditions")
except Exception as e:
    print(f"Could not plot band diagrams: {e}")

---

## 8. Exercises

### Exercise 1: Asymmetric Junction
Create a diode with different doping on each side (Na = 1e16, Nd = 1e18). How does this affect:
- The built-in potential?
- The depletion region distribution?
- The I-V characteristics?

In [ ]:
# Exercise 1: Asymmetric junction simulation
sim_asymmetric = create_pn_diode(
    length=2.0,
    junction_position=0.5,
    p_doping=1e16,   # Lightly doped P-side
    n_doping=1e18,   # Heavily doped N-side
    nx=200,
    ny=3,
    log_bands_eq=True,
    log_iv=True,
    forward_sweep=(0.0, 0.8, 0.02)
)

print("Running asymmetric junction simulation...")
result_asym = sim_asymmetric.run()

if result_asym.returncode == 0:
    # Calculate theoretical values
    Na_asym = 1e16
    Nd_asym = 1e18
    Vbi_asym = kT * np.log(Na_asym * Nd_asym / ni**2)
    
    # Depletion widths on each side
    xp = np.sqrt(2 * eps_si * Vbi_asym / q * Nd_asym / (Na_asym * (Na_asym + Nd_asym))) * 1e4
    xn = np.sqrt(2 * eps_si * Vbi_asym / q * Na_asym / (Nd_asym * (Na_asym + Nd_asym))) * 1e4
    
    print("\nAsymmetric Junction Analysis")
    print("="*40)
    print(f"Na = {Na_asym:.0e} cm^-3")
    print(f"Nd = {Nd_asym:.0e} cm^-3")
    print(f"Vbi = {Vbi_asym:.3f} V")
    print(f"Depletion in P-region: {xp:.4f} μm")
    print(f"Depletion in N-region: {xn:.4f} μm")
    print(f"Total depletion width: {xp + xn:.4f} μm")
    print(f"\nNote: Most depletion is in the lightly-doped P-region")
    
    # Plot band diagram
    sim_asymmetric.plot_band_diagram(title="Asymmetric PN Junction Band Diagram")

### Exercise 2: Temperature Dependence
Simulate the diode at different temperatures. How does the I-V curve change?

In [ ]:
# Exercise 2: Temperature dependence
temperatures = [300, 350, 400]
temp_results = {}

print("Simulating at different temperatures:")
print("="*50)

for T in temperatures:
    print(f"\nTemperature: {T} K")
    
    sim_T = create_pn_diode(
        length=2.0,
        junction_position=0.5,
        p_doping=1e17,
        n_doping=1e17,
        nx=150,
        ny=3,
        temperature=T,
        srh=True,
        log_iv=True,
        forward_sweep=(0.0, 0.8, 0.02)
    )
    
    result_T = sim_T.run()
    
    if result_T.returncode == 0:
        try:
            iv_data = sim_T.get_iv_data()
            V, I = iv_data.get_iv_data(electrode=2)
            temp_results[T] = (np.array(V), np.abs(np.array(I)))
            print(f"  Thermal voltage kT/q = {T * 8.617e-5:.4f} V")
        except:
            print(f"  Could not parse data")

# Plot temperature comparison
if temp_results:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for T, (V, I) in temp_results.items():
        ax.semilogy(V, I, linewidth=2, label=f'T = {T} K')
    
    ax.set_xlabel('Voltage (V)', fontsize=12)
    ax.set_ylabel('Current (A)', fontsize=12)
    ax.set_title('Temperature Effect on PN Diode I-V', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nKey observations:")
    print("- Higher temperature → Higher saturation current")
    print("- Turn-on voltage decreases with temperature")
    print("- Slope (q/nkT) decreases with temperature")

---

## Summary

In this notebook, you learned:

1. **PN Junction Physics**: Built-in potential, depletion region, I-V characteristics
2. **Running PADRE Simulations**: Using `create_pn_diode()` and `sim.run()`
3. **Band Diagrams**: Extracting and plotting band structure from PADRE output
4. **I-V Characteristics**: Forward and reverse bias behavior from simulation
5. **Parameter Extraction**: Ideality factor, saturation current from simulated data
6. **Parameter Effects**: How doping and temperature affect device behavior

**Key Equations:**
- Built-in potential: $V_{bi} = \frac{kT}{q}\ln\left(\frac{N_aN_d}{n_i^2}\right)$
- Ideal diode: $I = I_s(e^{qV/kT} - 1)$

**Next**: [03 - Schottky Diode](03_Schottky_Diode.ipynb) - Metal-semiconductor junctions